In [1]:
import numpy as np
class price_node:
    def __init__(self, step, S ,K):
        self.step = step
        self.S = S
        self.value = max(S-K,0)
        self.left = None
        self.right = None    
    def __repr__(self):
        return str('step:'+str(self.step)+'     S:'+str(self.S)+'   value:'+str(self.value))
    
class binomial_tree:
    def __init__(self, sigma = 0.1, r = 0.05, S0 = 10, K = 12, T = 3, dt =0.5,cp='call',ae='European'):
        self.n=int(T/dt)#terminal step
        self.root = price_node(0,S0,K)
        self.r=r;self.sigma=sigma
        self.S0=S0;self.K=K
        self.T=T;self.dt=dt
        self.u=np.exp(sigma*dt**0.5) ;self.d=np.exp(-sigma*dt**0.5)
        self.p=(np.exp(r*dt)-self.d)/(self.u-self.d)
        self.value=0
        self.n=int(T/dt)
        self.generate_tree(self.root)
    
    def generate_tree(self,node):
        if node.step==self.n:
            node.value=max(node.S-self.K,0)
        else:
            node.left=price_node(node.step+1,node.S*self.u,self.K)
            node.right=price_node(node.step+1,node.S*self.d,self.K)
            self.generate_tree(node.left)
            self.generate_tree(node.right)
            node.value=np.exp(-self.r*self.dt)*(self.p*node.left.value+(1-self.p)*node.right.value)

    def print_tree(self,node):
        if node is None:
            return
        print(node)
        self.print_tree(node.left)
        self.print_tree(node.right)

tree=binomial_tree(S0=810,K=800,T=0.5,dt=0.25,sigma=0.2,r=0.03)   

In [3]:
tree.print_tree(tree.root)

step:0     S:810   value:53.93134219594144
step:1     S:895.1884436412746   value:101.1659997859638
step:2     S:989.3362341097377   value:189.3362341097377
step:2     S:810.0   value:10.0
step:1     S:732.9183086091272   value:5.087690153119937
step:2     S:810.0   value:10.0
step:2     S:663.1719099931652   value:0
